In [1]:
#import libraries
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv("diabetes.csv")

In [3]:
df.head()

,Pat_Id,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,1,6,148,72,35,0,33.6,0.627,50,Diabetic
1,2,1,85,66,29,0,26.6,0.351,31,NotDiabetic
2,3,8,183,64,0,0,23.3,0.672,32,Diabetic
3,4,1,89,66,23,94,28.1,0.167,21,NotDiabetic
4,5,0,137,40,35,168,43.1,2.288,33,Diabetic


In [4]:
df['Outcome'].unique()

array(['Diabetic', 'NotDiabetic'], dtype=object)

In [5]:
df.drop(['Pat_Id', 'Pregnancies', 'SkinThickness', 'DiabetesPedigreeFunction','Age'], axis=1, inplace=True)

In [6]:
df['Outcome']=df['Outcome'].map({'Diabetic': 0, 'NotDiabetic': 1})

In [7]:
df.head()

,Glucose,BloodPressure,Insulin,BMI,Outcome
0,148,72,0,33.6,0
1,85,66,0,26.6,1
2,183,64,0,23.3,0
3,89,66,94,28.1,1
4,137,40,168,43.1,0


In [8]:
df['Outcome'].value_counts()

1    500
0    268
Name: Outcome, dtype: int64

In [9]:
X=df.drop("Outcome", axis=1)
y=df['Outcome']

In [10]:
X_train, X_test, y_train, y_test=train_test_split(X,y, test_size=0.2, random_state=400)

In [11]:
dummy=DummyClassifier()
dummy.fit(X_train, y_train)
pred=dummy.predict(X_test)
pred

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
      dtype=int64)

In [12]:
(y_test==pred).mean()

0.6298701298701299

In [13]:
accuracy_score(y_test, pred)

0.6298701298701299

In [14]:
print(classification_report(y_test, pred))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        57
           1       0.63      1.00      0.77        97

    accuracy                           0.63       154
   macro avg       0.31      0.50      0.39       154
weighted avg       0.40      0.63      0.49       154



In [15]:
log=LogisticRegression()
log.fit(X_train, y_train)
log_pred=log.predict(X_test)

In [16]:
print(classification_report(y_test, log_pred))

              precision    recall  f1-score   support

           0       0.80      0.56      0.66        57
           1       0.78      0.92      0.84        97

    accuracy                           0.79       154
   macro avg       0.79      0.74      0.75       154
weighted avg       0.79      0.79      0.78       154



In [17]:
import pickle

# saving the model in a pickle file
model = pickle.dump(log, open("model.pkl","wb"))

# loading the model int the console
model = pickle.load(open('model.pkl','rb'))


In [18]:
print(model.predict([[4.7,3.2,1.3,0.2]]))

[1]


In [ ]:
import numpy as np
from flask import Flask, request, render_template
import pickle

#Create an app object using the Flask class. 
app = Flask(__name__)

#Load the trained model. (Pickle file)
model = pickle.load(open('model.pkl', 'rb'))


@app.route('/')
def home():
    return render_template('index.html')


@app.route('/predict', methods=['POST'])
def predict():
    int_features = [float(x) for x in request.form.values()]
    features = [np.array(int_features)]
    prediction = model.predict(features)
    
    output = prediction[0]
    
    if output == 0:
        prediction_text = 'Diabetic'
    else:
        prediction_text = 'NotDiabetic'
    
    return render_template('index.html', prediction_text=prediction_text)


if __name__ == "__main__":
    app.run()

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [06/Mar/2024 14:39:58] "GET / HTTP/1.1" 200 -
[2024-03-06 14:40:10,942] ERROR in app: Exception on /predict [POST]
Traceback (most recent call last):
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 2447, in wsgi_app
    response = self.full_dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1952, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1821, in handle_user_exception
    reraise(exc_type, exc_value, tb)
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\_compat.py", line 39, in reraise
    raise value
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1950, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\ProgramData\Anaconda3\lib\site-packages\flask\app.py", line 1936, in dispatch_request
    return self.view_func